<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/Infer_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://akm-img-a-in.tosshub.com/indiatoday/images/story/201804/jet.jpeg
!wget https://tfhub.dev/tensorflow/lite-model/efficientnet/lite0/fp32/2?lite-format=tflite
!mv /content/2?lite-format=tflite efficientnet-lit0.tflite

--2020-08-28 13:50:35--  https://akm-img-a-in.tosshub.com/indiatoday/images/story/201804/jet.jpeg
Resolving akm-img-a-in.tosshub.com (akm-img-a-in.tosshub.com)... 104.84.160.48, 2600:1417:76:183::1ac0, 2600:1417:76:194::1ac0
Connecting to akm-img-a-in.tosshub.com (akm-img-a-in.tosshub.com)|104.84.160.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30622 (30K) [image/jpeg]
Saving to: ‘jet.jpeg’

jet.jpeg            100%[===================>]  29.90K  --.-KB/s    in 0.01s   

2020-08-28 13:50:36 (2.25 MB/s) - ‘jet.jpeg’ saved [30622/30622]

--2020-08-28 13:50:36--  https://tfhub.dev/tensorflow/lite-model/efficientnet/lite0/fp32/2?lite-format=tflite
Resolving tfhub.dev (tfhub.dev)... 108.177.125.138, 108.177.125.113, 108.177.125.102, ...
Connecting to tfhub.dev (tfhub.dev)|108.177.125.138|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/tfhub-lite-models/tensorflow/lite-model/efficientnet/lite0/fp32/

In [ ]:
import math
import tensorflow as tf

import time
import os
import numpy as np
import PIL
import PIL.Image as Image

import argparse
model_path ="./efficientnet-lit0.tflite"
input_file ="jet.jpeg"

input_mean =127.5
input_std  =127.5

        

def infer_tflite_model():

  interpreter = tf.lite.Interpreter(model_path=model_path)
  interpreter.allocate_tensors()
    
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  print(output_details)

  # check the type of the input tensor
  floating_model = input_details[0]['dtype'] == np.float32

  # NxHxWxC, H:1, W:2
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]
  img = Image.open(input_file).resize((width, height))

  # add N dim
  input_data = np.expand_dims(img, axis=0)

  if floating_model:
    input_data = (np.float32(input_data) - input_mean) / input_std

  interpreter.set_tensor(input_details[0]['index'], input_data)

  interpreter.invoke()

  output_data = interpreter.get_tensor(output_details[0]['index'])
  results = np.squeeze(output_data)

  top_k = results.argsort()[-5:][::-1]
  print(top_k)

                                       
infer_tflite_model()



[{'name': 'Softmax', 'index': 163, 'shape': array([   1, 1000], dtype=int32), 'shape_signature': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[895 403 744 657  51]
